In [47]:
import json
import emoji
import telegram
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import time
from bs4 import BeautifulSoup
import random
from html import unescape

In [48]:
ID = ''
PW = ''
driver = webdriver.Chrome()
driver.get('https://everytime.kr/login')
driver.find_element(By.NAME, 'id').send_keys(ID)
driver.find_element(By.NAME, 'password').send_keys(PW)
driver.find_element(By.XPATH, '//input[@value="에브리타임 로그인"]').click()

In [49]:
def extract_text(tags):
    """
    주어진 BeautifulSoup 태그 리스트에서 텍스트를 추출하여 연결합니다.
    
    :param tags: BeautifulSoup 객체들의 리스트
    :return: 연결된 텍스트 문자열
    """
    full_text = ""
    for tag in tags:
        # 태그에서 텍스트 추출
        text = tag.get_text()

        # HTML 엔티티를 일반 텍스트로 변환 (예: &amp; -> &)
        text = unescape(text)  # 이 부분에서 오류가 발생했습니다.

        # 공백 제거 및 정리
        text = text.strip()

        # 텍스트 추가
        full_text += text + " "  # 단어 사이에 공백 추가

    return full_text.strip()  # 마지막 공백 제거

In [50]:
def extract_comments(soup):
    comments = []
    last_parent_author = None

    for article in soup.find_all('article'):
        comment_type = None  # 여기에 초기값 설정
        author = ""
        comment_text = ""
        timestamp = ""
        vote_count = 0

        h3_tag = article.find('h3', class_='medium')
        p_tag = article.find('p', class_='large')
        time_tag = article.find('time', class_='medium')
        vote_tag = article.find('li', class_='vote')

        if h3_tag:
            author = h3_tag.text.strip()
        if p_tag:
            comment_text = p_tag.text.strip()
        if time_tag:
            timestamp = time_tag.text.strip()
        if vote_tag:
            vote_count = vote_tag.text.strip()

        if 'parent' in article['class']:
            last_parent_author = author
            comment_type = 'parent'
        elif 'child' in article['class']:
            comment_type = 'child'

        if comment_type:
            comment_dict = {
                "Type": comment_type,
                "Author": author,
                "Comment": comment_text,
                "Timestamp": timestamp,
                "Vote Count": vote_count
            }

            # 대댓글일 경우 부모 댓글의 작성자 추가
            if comment_type == 'child' and last_parent_author:
                comment_dict["Parent Author"] = last_parent_author

            comments.append(comment_dict)

    return comments


In [51]:
result_dict = {
    "title": [],
    "detail": [],
    "comments": [],  # 'comments' 키를 추가
    "url": [],
    "timestamp": []
}


In [52]:
file_path = "url_test.json"
test = []
with open(file_path, 'r', encoding='UTF-8') as json_file:
    data = json.load(json_file)

for url in data:
    driver.get("https://" + url)
    time.sleep(3 + random.uniform(1, 5))
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    
    title = extract_text(soup.find_all('h2', class_='large'))
    detail = extract_text(soup.find('p', class_='large'))

    comments = extract_comments(soup)

    result_dict["title"].append(title)
    result_dict["detail"].append(detail)
    result_dict["url"].append(url)
    result_dict["comments"].append(comments)

In [53]:
result_dict

{'title': ['[❗️D-6❗️] 🟢ICPC Sinchon 알고리즘 캠프 참가자 모집🟢',
  '설/카 대학원 진학시 학점',
  '타과생은',
  '복전생은 이산구조',
  ''],
 'detail': ['안녕하세요. 신촌지역 대학교 프로그래밍 동아리 연합입니다: https://icpc-sinchon.io.  ICPC Sinchon은 5개의 신촌연합 대학교(서강, 숙명, 연세, 이화, 홍익) 프로그래밍 동아리가 모여 매 여름/겨울방학마다 알고리즘 캠프 및 SUAPC를 개최하고 있습니다.   아래는 캠프 관련 상세사항이니 숙지 후 신청해주세요!   🗓️ 모집 일정   24. 01. 05 ~ 24. 01. 21   📋 캠프 신청대상 및 일정   C++을 이해할 수 있는, 신촌연합 소속 학생이라면 누구나 (재/휴학 무관)   📌1/22(1회차) ~ 2/22(10회차) 동안  📌월/목요일 18시에 커리큘럼별 일정에 따라 대면/비대면으로 알고리즘 캠프 진행  대면 : 엘리스 Lab 성수점 / 비대면 : 디스코드   👨🏻\u200d🏫강사진 안내  초급:  박찬솔, 채성우, 황채원  중급 : 김건우, 김영현, 김태윤   💰💰캠프 수강비 안내💰💰  캠프를 열심히 듣고 주어진 과제를 다 완료하면 낸 돈을 모두 환급받을 수 있는 환급옵션과 비환급 옵션이 있습니다.   1) 환급옵션 6만원  : 10회 중 9회이상 강의에 출석 시 1만원 환급  : 필수문제 해결시 전체문제에 대한 푼 문제 비율로 5만원 일부 혹은 전체 환급   2) 비환급 옵션  3만원  : 환급받는 금액없이 3만원을 납부하시면 됩니다.   💥캠프 주의사항💥  ☝️초급/중급 코스 중 하나만 선택 (중복 불가능)  ✌️각 코스에 따른 대면일정과 강사진이 다르니 아래 세부사항 꼭 확인   [초급 수강권장 대상]  알고리즘을 처음 접하거나, 기초 알고리즘을 이론과 문제로 확실히 잡고 싶으신 분   [초급 커리큘럼]  1회차(채성우): Problem Solving, Time complexity, Basic C++

In [54]:
file_path="everytime_computer_data.json"
with open(file_path, 'w', encoding='UTF-8') as json_file:
    json.dump(result_dict, json_file, indent=2, ensure_ascii=False)
